In [2]:
import json

r = []
with open('imda-science-word-timestamp.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        r.append(l)

In [5]:
from tqdm import tqdm
import copy
import re

results = []
for r_ in tqdm(r):
    r_ = copy.copy(r_)
    r_['audio_filename'] = r_['audio_filename'].replace('/home/husein/ssd3/', '').replace('/home/husein/ssd4/', '')
    clean_text = re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip().split()
    ratio = (len([w for w in clean_text if len(w) <= 1]) / len(clean_text))
    if ratio > 0.2:
        continue
    results.append(r_)

100%|████████████████████████████████████████████████████████████████████████████████████| 2775149/2775149 [00:07<00:00, 348435.47it/s]


In [6]:
from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [7]:
filtered = []
for r_ in tqdm(results):
    if skip_trigrams(re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip()):
        continue
    filtered.append(r_)
len(filtered)

100%|████████████████████████████████████████████████████████████████████████████████████| 2689940/2689940 [00:14<00:00, 179756.32it/s]


1960115

In [8]:
for r_ in filtered:
    if len(r_['word_timestamp']) > 2000:
        print(r_)
        break

{'audio_filename': 'prepared-science-chunks/755-0.mp3', 'word_timestamp': '<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> vasectomy<|0.40|><|0.58|> that<|0.66|><|0.72|> takes<|0.92|><|1.04|> less<|1.18|><|1.26|> than<|1.34|><|1.36|> 30<|1.38|><|1.66|> minutes,<|1.92|><|2.32|> and<|2.38|><|2.42|> the<|2.48|><|2.54|> next,<|2.82|><|3.04|> a<|3.04|><|3.22|> complex<|3.62|><|3.74|> bladder<|4.00|><|4.10|> reconstruction<|4.64|><|4.74|> lasting<|5.08|><|5.24|> several<|5.54|><|5.68|> hours.<|6.04|><|6.50|> Some<|6.62|><|6.70|> urologists<|7.18|><|7.36|> may<|7.46|><|7.52|> enjoy<|7.76|><|7.82|> the<|7.88|><|8.02|> complex<|8.56|><|8.86|> life<|9.04|><|9.12|> or<|9.16|><|9.20|> death<|9.36|><|9.42|> surgeries<|9.84|><|9.98|> where<|10.16|><|10.32|> you<|10.38|><|10.52|> carefully<|10.96|><|11.02|> dissect<|11.30|><|11.46|> around<|11.80|><|11.84|> the<|11.90|><|11.98|> aorta<|12.32|><|12.58|> and<|12.68|><|12.76|> vena<|12.94|><|13.04|> cava,<|13.26|><|13.62|> removing<|13.96|><|14

In [9]:
len(filtered) / len(results)

0.7286835394098009

In [10]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('pseudolabel-science-word-timestamp.parquet')

In [11]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-science-word-timestamp.parquet",
    path_in_repo="data/science_context-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

pseudolabel-science-word-timestamp.parquet:   0%|          | 0.00/550M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/41006e76a0a40e17752384019c2a18e11d8ff1ce', commit_message='Upload data/science_context-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='41006e76a0a40e17752384019c2a18e11d8ff1ce', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)

In [3]:
!rm *.parquet

In [4]:
!rm imda-science-word-timestamp.jsonl

In [5]:
!du -hs prepared-science-chunks

84G	prepared-science-chunks
